# Tutorial 2: Edge & Line Detection Pipelines

This tutorial covers the full edge and line segment detection pipeline using the
**le_edge** and **le_lsd** Python modules from the LineExtraction library.

## What You Will Learn

1. **Edge Sources** — compute gradients, direction maps, and seeds
2. **Non-Maximum Suppression (NMS)** — standalone NMS usage
3. **Edge Segment Detectors** — EsdSimple, EsdDrawing, EsdLinking, EsdPattern
4. **Edge Segment Properties & Flags** — EdgeSegment, direction/quadrature options
5. **Line Segment Detectors (LSD)** — all 9 detectors with their parameters
6. **Grand Comparison** — side-by-side comparison of all detectors
7. **Auxiliary Image Data** — accessing internal detector data layers
8. **Line Optimization** — refining detected segments
9. **Noise Robustness** — detector behavior under noise

## Prerequisites

- Completed **Tutorial 1: Library Fundamentals** (or equivalent knowledge)
- Built the LE Python bindings: `bazel build //libs/...`
- Python kernel set to the project `.venv`

## 1. Setup & Imports

In [ ]:
import sys, pathlib

# --- Locate workspace root and add Bazel output dirs to sys.path ---
workspace = pathlib.Path.cwd()
while not (workspace / "MODULE.bazel").exists():
    if workspace == workspace.parent:
        raise RuntimeError("Cannot find LineExtraction workspace root (MODULE.bazel)")
    workspace = workspace.parent

for lib in ["imgproc", "edge", "geometry", "eval", "lsd"]:
    p = workspace / f"bazel-bin/libs/{lib}/python"
    if p.exists():
        sys.path.insert(0, str(p))
    else:
        print(f"Warning: Not found: {p}  — run: bazel build //libs/{lib}/...")

sys.path.insert(0, str(workspace / "python"))

import numpy as np
import matplotlib.pyplot as plt
import timeit
%matplotlib inline

# NOTE: Do NOT import cv2 — the LE bindings ship their own statically linked
# OpenCV build. Using pip's cv2 would cause symbol conflicts.

import le_imgproc
import le_edge
import le_lsd
import le_geometry
from lsfm.data import TestImages

print(f"Workspace: {workspace}")
print(f"le_edge loaded:  {len([x for x in dir(le_edge) if not x.startswith('_')])} symbols")
print(f"le_lsd loaded:   {len([x for x in dir(le_lsd) if not x.startswith('_')])} symbols")

In [ ]:
# --- Common helpers ---

def show_images(images, titles, cmap="gray", figsize=None):
    """Show a list of images side by side."""
    n = len(images)
    if figsize is None:
        figsize = (4 * n, 4)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img, cmap=cmap if img.ndim == 2 else None)
        ax.set_title(title, fontsize=10)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

def show_grid(images, titles, cols=3, cmap="gray", figsize=None):
    """Show images in a grid layout."""
    n = len(images)
    rows = (n + cols - 1) // cols
    if figsize is None:
        figsize = (4 * cols, 4 * rows)
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = np.array(axes).flatten()
    for i, (ax, img, title) in enumerate(zip(axes, images, titles)):
        ax.imshow(img, cmap=cmap if img.ndim == 2 else None)
        ax.set_title(title, fontsize=10)
        ax.axis("off")
    for ax in axes[n:]:
        ax.axis("off")
    plt.tight_layout()
    plt.show()

def make_test_image(size=200):
    """Create a uint8 grayscale image with rectangles and diagonal lines."""
    img = np.zeros((size, size), dtype=np.uint8)
    s = size
    img[s//5:4*s//5, s//5:4*s//5] = 200
    img[2*s//5:3*s//5, 2*s//5:3*s//5] = 80
    for i in range(s//5, 4*s//5):
        j = s//5 + (i - s//5)
        if 0 <= j < s:
            img[i, j] = 255
    return img

def bgr_to_rgb(img):
    """Convert BGR image (from LE drawing) to RGB for matplotlib."""
    return img[:, :, ::-1]

def idx_to_rc(linear_indices, cols):
    """Convert linear pixel indices to (row, col) arrays."""
    indices = np.array(linear_indices)
    return indices // cols, indices % cols

print("Helpers ready.")

In [ ]:
# --- Load images ---
ti = TestImages()

windmill = plt.imread(str(ti.windmill()))
# Convert to grayscale via luminance formula (ensure writable copy)
if windmill.ndim == 3:
    windmill_gray = np.dot(windmill[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)
else:
    windmill_gray = np.ascontiguousarray(windmill.copy())

# Load a BSDS500 image for variety
bsds_iter = ti.bsds500()
bsds_path = next(bsds_iter)
bsds_img = plt.imread(str(bsds_path))
if bsds_img.ndim == 3:
    bsds_gray = np.dot(bsds_img[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)
else:
    bsds_gray = np.ascontiguousarray(bsds_img.copy())

# Also create a synthetic test image
synth = make_test_image(200)

show_images([windmill_gray, bsds_gray, synth],
            [f"Windmill {windmill_gray.shape}", f"BSDS500 {bsds_gray.shape}", "Synthetic"])
print("Images loaded.")

---
## 2. Edge Sources

An **EdgeSource** wraps gradient computation, non-maximum suppression, and
hysteresis thresholding into a single pipeline. Three gradient operators are
available: `EdgeSourceSobel`, `EdgeSourceScharr`, and `EdgeSourcePrewitt`.

Each edge source provides:
- `magnitude()`, `direction()`, `gx()`, `gy()` — raw gradient data
- `direction_map()` — quantized direction map (int8)
- `seeds()` — strong edge pixel indices after NMS
- `hysteresis()` — full hysteresis edge map
- `hysteresis_binary()` — binary edge map
- `hysteresis_edgels()` — list of edge pixel indices

In [ ]:
# Process the synthetic image with all three edge sources
edge_sources = {
    "Sobel": le_edge.EdgeSourceSobel(),
    "Scharr": le_edge.EdgeSourceScharr(),
    "Prewitt": le_edge.EdgeSourcePrewitt(),
}

for name, es in edge_sources.items():
    es.process(synth)

# Compare magnitude output
show_images(
    [es.magnitude() for es in edge_sources.values()],
    [f"{name} Magnitude" for name in edge_sources],
)

# Compare direction maps (quantized to int8: 0=none, ±1..±4 = directions)
show_images(
    [es.direction_map().astype(np.float32) for es in edge_sources.values()],
    [f"{name} Direction Map" for name in edge_sources],
    cmap="coolwarm",
)

In [ ]:
# Detailed look at EdgeSourceSobel outputs
es = le_edge.EdgeSourceSobel()
es.process(synth)

print(f"Gradient components shape: gx={es.gx().shape}, gy={es.gy().shape}")
print(f"Magnitude max: {es.magnitude_max():.2f}")
print(f"Magnitude threshold (0.5): {es.magnitude_threshold(0.5):.2f}")
print(f"Direction range: [{es.direction_range().lower:.4f}, {es.direction_range().upper:.4f}]")
print(f"Number of seeds: {len(es.seeds())}")

show_images(
    [es.gx().astype(np.float32), es.gy().astype(np.float32),
     es.magnitude(), es.direction()],
    ["Gx", "Gy", "Magnitude", "Direction"],
)

In [ ]:
# Hysteresis thresholding produces different edge representations
es = le_edge.EdgeSourceSobel()
es.process(windmill_gray)

hyst = es.hysteresis()
hyst_bin = es.hysteresis_binary()
edgels = es.hysteresis_edgels()

print(f"Hysteresis map dtype: {hyst.dtype}, unique values: {len(np.unique(hyst))}")
print(f"Binary map dtype: {hyst_bin.dtype}")
print(f"Number of edge pixels: {len(edgels)}")

show_images(
    [windmill_gray, hyst, hyst_bin],
    ["Original", "Hysteresis", "Binary"],
)

In [ ]:
# Seeds visualization: seeds are strong-edge pixel indices after NMS.
# They serve as starting points for edge segment tracing.
es = le_edge.EdgeSourceSobel()
es.process(synth)

seeds = es.seeds()
rows, cols_arr = idx_to_rc(seeds, synth.shape[1])

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(synth, cmap="gray")
axes[0].set_title("Original")
axes[1].imshow(synth, cmap="gray", alpha=0.4)
axes[1].scatter(cols_arr, rows, c="red", s=1, marker=".")
axes[1].set_title(f"Seeds ({len(seeds)} pixels)")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Exercise 2.1

Process the **windmill image** with all three edge sources. Compare the number
of seeds each one produces and visualize the hysteresis binary maps side by side.
Which operator produces the most edges?

In [ ]:
# TODO: Create all three edge sources, process windmill_gray,
#       print the seed count for each, and show hysteresis_binary() maps.

**Solution**

In [ ]:
sources = {
    "Sobel": le_edge.EdgeSourceSobel(),
    "Scharr": le_edge.EdgeSourceScharr(),
    "Prewitt": le_edge.EdgeSourcePrewitt(),
}

for name, src in sources.items():
    src.process(windmill_gray)
    print(f"{name:8s}: {len(src.seeds()):5d} seeds,  mag_max = {src.magnitude_max():.1f}")

show_images(
    [src.hysteresis_binary() for src in sources.values()],
    [f"{name} Binary Edges" for name in sources],
)

Scharr typically produces the most edges because its kernel has higher angular
accuracy and slightly higher response. Prewitt produces the fewest because
its kernel has the smallest support.

---
## 3. Non-Maximum Suppression (NMS)

While `EdgeSource` includes NMS internally, you can also use
`NonMaximaSuppression` standalone. This is useful when you want to
compute gradients with one tool (e.g., `le_imgproc.SobelGradient`) and
apply NMS separately.

**Constructor:** `NonMaximaSuppression(th_low=0.004, th_high=0.012, border=1)`

- `th_low` / `th_high`: hysteresis thresholds (relative to magnitude range)
- `border`: pixels to skip at image boundary

In [ ]:
# Step 1: Compute gradient using le_imgproc
grad = le_imgproc.SobelGradient()
grad.process(synth)

# Step 2: Apply NMS manually
nms = le_edge.NonMaximaSuppression(low=0.004, high=0.012, border=1)

gx = grad.gx()
gy = grad.gy()
mag = grad.magnitude()
low = np.float32(grad.magnitude_threshold(0.004))
high = np.float32(grad.magnitude_threshold(0.012))

nms.process(gx, gy, mag, low, high)

print(f"NMS name: {nms.name()}")
print(f"Thresholds: low={nms.threshold_low():.4f}, high={nms.threshold_high():.4f}")
print(f"Border: {nms.border()}")
print(f"Seeds found: {len(nms.seeds())}")

# Visualize direction map from NMS
dmap = nms.direction_map()
show_images(
    [mag, dmap.astype(np.float32)],
    ["Gradient Magnitude", "NMS Direction Map"],
    cmap="coolwarm",
)

In [ ]:
# Effect of threshold tuning on seed count
grad = le_imgproc.SobelGradient()
grad.process(windmill_gray)
gx, gy, mag = grad.gx(), grad.gy(), grad.magnitude()

th_values = [0.001, 0.004, 0.01, 0.02, 0.05, 0.1]
seed_counts = []
for th in th_values:
    nms = le_edge.NonMaximaSuppression(low=th, high=th * 3)
    low = np.float32(grad.magnitude_threshold(th))
    high = np.float32(grad.magnitude_threshold(th * 3))
    nms.process(gx, gy, mag, low, high)
    seed_counts.append(len(nms.seeds()))

plt.figure(figsize=(8, 4))
plt.plot(th_values, seed_counts, "o-")
plt.xlabel("NMS threshold (low)")
plt.ylabel("Number of seeds")
plt.title("NMS Threshold vs. Seed Count")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ValueManager for NMS
nms = le_edge.NonMaximaSuppression()
vals = nms.values()
print("NMS parameters:")
for name, val in vals.items():
    print(f"  {name}: {val}")

# Modify thresholds at runtime
nms.set_threshold(0.02, 0.08)
print(f"\nAfter set_threshold: low={nms.threshold_low()}, high={nms.threshold_high()}")

---
## 4. Edge Segment Detectors

Edge segment detectors trace connected edge chains from the output of an edge
source. Four algorithms are available, each with different tracing strategies:

| Detector | Algorithm | Key Parameters | Gap Bridging |
|----------|-----------|----------------|--------------|
| `EsdSimple` | Basic following | `min_pixels` | No |
| `EsdDrawing` | Edge Drawing | `min_pixels`, `mag_mul`, `mag_th` | No |
| `EsdLinking` | Link-based | `min_pixels`, `max_gap`, `mag_mul`, `mag_th` | Yes |
| `EsdPattern` | Pattern match | `min_pixels`, `max_gap`, `mag_mul`, `mag_th`, `pat_tol` | Yes |

All detectors output:
- `segments()` → list of `EdgeSegment` (begin/end indices into points array)
- `points()` → flat list of linear pixel indices

### 4.1 EsdSimple — Baseline Detector

In [ ]:
# Prepare the edge source
es = le_edge.EdgeSourceSobel()
es.process(synth)

# EsdSimple: the simplest approach — follow connected edge pixels
esd = le_edge.EsdSimple(min_pixels=5)
esd.detect(es)

segments = esd.segments()
points = esd.points()

print(f"EsdSimple: {len(segments)} segments, {len(points)} total points")

# Visualize segments in different colors
viz = np.zeros((*synth.shape, 3), dtype=np.uint8)
for seg in segments:
    seg_pts = points[seg.begin:seg.end]
    r, c = idx_to_rc(seg_pts, synth.shape[1])
    color = np.random.randint(50, 255, size=3)
    viz[r, c] = color

show_images([synth, viz], ["Original", f"EsdSimple ({len(segments)} segments)"])

### 4.2 EsdDrawing — Edge Drawing Algorithm

In [ ]:
esd = le_edge.EsdDrawing(min_pixels=5, mag_mul=2.0, mag_th=3.0)
esd.detect(es)

segments = esd.segments()
points = esd.points()
print(f"EsdDrawing: {len(segments)} segments, {len(points)} total points")
print(f"  Name: {esd.name()}")
print(f"  Parameters: {esd.values()}")

# Visualize
viz = np.zeros((*synth.shape, 3), dtype=np.uint8)
for seg in segments:
    seg_pts = points[seg.begin:seg.end]
    r, c = idx_to_rc(seg_pts, synth.shape[1])
    color = np.random.randint(50, 255, size=3)
    viz[r, c] = color

show_images([synth, viz], ["Original", f"EsdDrawing ({len(segments)} segs)"])

### 4.3 EsdLinking — Gap-Bridging Detection

In [ ]:
# EsdLinking can bridge small gaps in edge contours
# Create edge source first for this cell
es_link = le_edge.EdgeSourceSobel()
es_link.process(synth)

esd_link = le_edge.EsdLinking(min_pixels=5, max_gap=2, mag_mul=2.0, mag_th=3.0)
esd_link.detect(es_link)

segs_link = esd_link.segments()
pts_link = esd_link.points()
print(f"EsdLinking: {len(segs_link)} segments, {len(pts_link)} total points")

# Compare effect of max_gap
for gap in [0, 1, 2, 3, 5]:
    esd_g = le_edge.EsdLinking(min_pixels=5, max_gap=gap)
    esd_g.detect(es_link)
    print(f"  max_gap={gap}: {len(esd_g.segments())} segments")

### 4.4 EsdPattern — Pattern-Based Detection

In [ ]:
# EsdPattern uses pattern matching for edge following
esd_pat = le_edge.EsdPattern(min_pixels=5, max_gap=2, mag_mul=2.0, mag_th=3.0, pat_tol=1)
esd_pat.detect(es)

segs_pat = esd_pat.segments()
pts_pat = esd_pat.points()

print(f"EsdPattern: {len(segs_pat)} segments, {len(pts_pat)} total points")

# EsdPattern has extra accessors: patterns() and pattern_segments()
patterns = esd_pat.patterns()
pat_segs = esd_pat.pattern_segments()
print(f"  Patterns: {len(patterns)}")
print(f"  Pattern segments: {len(pat_segs)}")

### 4.5 Comparison of All Edge Segment Detectors

In [ ]:
# Compare all 4 detectors on the windmill image
es_wm = le_edge.EdgeSourceSobel()
es_wm.process(windmill_gray)

detectors = {
    "EsdSimple": le_edge.EsdSimple(min_pixels=10),
    "EsdDrawing": le_edge.EsdDrawing(min_pixels=10),
    "EsdLinking": le_edge.EsdLinking(min_pixels=10, max_gap=2),
    "EsdPattern": le_edge.EsdPattern(min_pixels=10, max_gap=2),
}

images, titles = [], []
for name, esd in detectors.items():
    esd.detect(es_wm)
    segs = esd.segments()
    pts = esd.points()
    
    viz = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
    for seg in segs:
        seg_pts = pts[seg.begin:seg.end]
        r, c = idx_to_rc(seg_pts, windmill_gray.shape[1])
        color = np.random.randint(50, 255, size=3)
        viz[r, c] = color
    
    images.append(viz)
    titles.append(f"{name}\n({len(segs)} segs)")
    print(f"{name:12s}: {len(segs):4d} segments, {len(pts):6d} points")

show_grid(images, titles, cols=2, figsize=(12, 10))

In [ ]:
# Alternative: detect from raw data instead of edge source
# This is useful when you compute NMS externally.
esd_raw = le_edge.EsdDrawing(min_pixels=5)
esd_raw.detect(es.direction_map(), es.magnitude(), es.seeds())

print(f"Detect from raw data: {len(esd_raw.segments())} segments")

### 4.6 EdgeSegment Properties

In [ ]:
# EdgeSegment stores begin/end indices into the points array, plus metadata flags
seg = le_edge.EdgeSegment(10, 25, le_edge.ES_REVERSE, 42)

print(f"begin={seg.begin}, end={seg.end}, size={seg.size()}")
print(f"id={seg.id}")
print(f"flags={seg.flags}")
print(f"reverse={seg.reverse()}, closed={seg.closed()}")
print(f"first()={seg.first()}, last()={seg.last()}")

# For a reversed segment, first() returns the last index and vice versa
seg_fwd = le_edge.EdgeSegment(0, 10)
seg_rev = le_edge.EdgeSegment(0, 10, le_edge.ES_REVERSE)
print(f"\nForward: first={seg_fwd.first()}, last={seg_fwd.last()}")
print(f"Reverse: first={seg_rev.first()}, last={seg_rev.last()}")

# Combine flags
seg_both = le_edge.EdgeSegment(0, 10, le_edge.ES_REVERSE | le_edge.ES_CLOSED)
print(f"\nCombined flags: reverse={seg_both.reverse()}, closed={seg_both.closed()}")

In [ ]:
# Direction and Quadrature option enums
print("Direction Options:")
print(f"  NONE = {le_edge.ESDirectionOptions.NONE}")
print(f"  GXGY = {le_edge.ESDirectionOptions.GXGY}")
print(f"  DIR  = {le_edge.ESDirectionOptions.DIR}")

print("\nQuadrature Options:")
print(f"  MAG    = {le_edge.ESQuadratureOptions.MAG}")
print(f"  ENERGY = {le_edge.ESQuadratureOptions.ENERGY}")
print(f"  PC     = {le_edge.ESQuadratureOptions.PC}")

### Exercise 2.2

Build a full edge detection pipeline:
1. Use `EdgeSourceScharr` on the windmill image
2. Detect segments with `EsdLinking` using `max_gap=3`
3. Find the **longest** segment (largest `.size()`)
4. Visualize only that longest segment highlighted in red on the original image

In [ ]:
# TODO: Build the pipeline, find the longest segment, visualize it.

**Solution**

In [ ]:
es = le_edge.EdgeSourceScharr()
es.process(windmill_gray)

esd = le_edge.EsdLinking(min_pixels=5, max_gap=3)
esd.detect(es)

segments = esd.segments()
points = esd.points()

# Find the longest segment
longest = max(segments, key=lambda s: s.size())
print(f"Longest segment: id={longest.id}, size={longest.size()} pixels")
print(f"  begin={longest.begin}, end={longest.end}")
print(f"  closed={longest.closed()}, reverse={longest.reverse()}")

# Visualize: all segments in gray, longest in red
viz = np.stack([windmill_gray]*3, axis=-1).copy()
for seg in segments:
    seg_pts = points[seg.begin:seg.end]
    r, c = idx_to_rc(seg_pts, windmill_gray.shape[1])
    viz[r, c] = [100, 100, 100]

longest_pts = points[longest.begin:longest.end]
r, c = idx_to_rc(longest_pts, windmill_gray.shape[1])
viz[r, c] = [255, 0, 0]

show_images([windmill_gray, viz],
            ["Original", f"Longest segment: {longest.size()} px (red)"])

The longest segment is highlighted in red. In structural images like windmill,
the longest edge segment typically follows a prominent contour boundary.

---
## 5. Line Segment Detectors — Overview

The `le_lsd` module provides **9 different line segment detection algorithms**.
They all share a common interface:

```python
detector = le_lsd.LsdXXX(...)   # construct with parameters
detector.detect(gray_image)      # run detection on uint8 grayscale
segments = detector.line_segments()  # list of LineSegment objects
endpoints = detector.end_points()    # list of (x1,y1,x2,y2) tuples
lines = detector.lines()             # list of Line objects
```

### Detector Categories

| Category | Detectors | Strategy |
|----------|-----------|----------|
| **Region-based** | `LsdCC`, `LsdCP`, `LsdBurns`, `LsdFBW` | Group gradient-aligned pixels |
| **Edge-based** | `LsdEDLZ`, `LsdEL`, `LsdEP` | Trace edges, then fit lines |
| **Other** | `LsdFGioi`, `LsdHoughP` | NFA validation / Hough voting |

### 5.1 LsdCC — Connected Components

In [ ]:
det = le_lsd.LsdCC()
det.detect(synth)

segments = det.line_segments()
print(f"LsdCC: {len(segments)} line segments detected")
print(f"Type: {type(det).__name__}")

# Draw detected segments
canvas = np.zeros((*synth.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas, segments)

show_images([synth, bgr_to_rgb(canvas)],
            ["Original", f"LsdCC ({len(segments)} lines)"])

In [ ]:
# LsdCC flags control behavior
print("LsdCC flag constants:")
print(f"  CC_FIND_NEAR_COMPLEX = {le_lsd.CC_FIND_NEAR_COMPLEX}")
print(f"  CC_CORNER_RULE       = {le_lsd.CC_CORNER_RULE}")
print(f"  CC_ADD_THICK_PIXELS  = {le_lsd.CC_ADD_THICK_PIXELS}")

# Compare with and without flags on windmill
det_default = le_lsd.LsdCC()
det_default.detect(windmill_gray)
segs_default = det_default.line_segments()

det_corner = le_lsd.LsdCC(flags=le_lsd.CC_CORNER_RULE)
det_corner.detect(windmill_gray)
segs_corner = det_corner.line_segments()

det_all = le_lsd.LsdCC(flags=le_lsd.CC_CORNER_RULE | le_lsd.CC_ADD_THICK_PIXELS)
det_all.detect(windmill_gray)
segs_all = det_all.line_segments()

print(f"\nDefault:       {len(segs_default)} segments")
print(f"Corner rule:   {len(segs_corner)} segments")
print(f"Corner+Thick:  {len(segs_all)} segments")

### 5.2 LsdCP — Connected Components with Pattern Linking

In [ ]:
det = le_lsd.LsdCP()
det.detect(windmill_gray)
segments = det.line_segments()

canvas = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas, segments)

print(f"LsdCP: {len(segments)} segments")
show_images([windmill_gray, bgr_to_rgb(canvas)],
            ["Original", f"LsdCP ({len(segments)} lines)"])

### 5.3 LsdBurns — Classic Burns Algorithm

In [ ]:
det = le_lsd.LsdBurns()
det.detect(windmill_gray)
segments = det.line_segments()

canvas = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas, segments)

print(f"LsdBurns: {len(segments)} segments")
print(f"  Flag: BURNS_NMS = {le_lsd.BURNS_NMS}")

# Compare with/without NMS
det_nms = le_lsd.LsdBurns(flags=le_lsd.BURNS_NMS)
det_nms.detect(windmill_gray)
print(f"  With NMS: {len(det_nms.line_segments())} segments")

show_images([windmill_gray, bgr_to_rgb(canvas)],
            ["Original", f"LsdBurns ({len(segments)} lines)"])

### 5.4 LsdFBW — Fast Region Growing

In [ ]:
det = le_lsd.LsdFBW()
det.detect(windmill_gray)
segments = det.line_segments()

canvas = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas, segments)

print(f"LsdFBW default:         {len(segments)} segments")

# FBW flags
print(f"\nFBW flags: FBW_NMS={le_lsd.FBW_NMS}, FBW_PATAN={le_lsd.FBW_PATAN}")

det_nms = le_lsd.LsdFBW(flags=le_lsd.FBW_NMS)
det_nms.detect(windmill_gray)
print(f"With FBW_NMS:           {len(det_nms.line_segments())} segments")

det_patan = le_lsd.LsdFBW(flags=le_lsd.FBW_PATAN)
det_patan.detect(windmill_gray)
print(f"With FBW_PATAN:         {len(det_patan.line_segments())} segments")

show_images([windmill_gray, bgr_to_rgb(canvas)],
            ["Original", f"LsdFBW ({len(segments)} lines)"])

### Exercise 2.3

Compare `LsdCC` and `LsdBurns` with custom parameters on the windmill:

1. Create `LsdCC` with `th_low=0.01, th_high=0.03, min_pix=10, max_gap=2, err_dist=2.0`
2. Create `LsdBurns` with `th_low=0.01, th_high=0.03, min_pix=10, part_num=16`
3. Run both on the windmill, draw results side-by-side
4. Print the average segment length for each detector

In [ ]:
# TODO: Create detectors with custom params, detect, compare visually and numerically.

**Solution**

In [ ]:
det_cc = le_lsd.LsdCC(th_low=0.01, th_high=0.03, min_pix=10, max_gap=2, err_dist=2.0)
det_cc.detect(windmill_gray)
segs_cc = det_cc.line_segments()

det_burns = le_lsd.LsdBurns(th_low=0.01, th_high=0.03, min_pix=10, part_num=16)
det_burns.detect(windmill_gray)
segs_burns = det_burns.line_segments()

# Visualize
canvas_cc = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas_cc, segs_cc)

canvas_burns = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas_burns, segs_burns)

show_images(
    [bgr_to_rgb(canvas_cc), bgr_to_rgb(canvas_burns)],
    [f"LsdCC ({len(segs_cc)} segs)", f"LsdBurns ({len(segs_burns)} segs)"],
)

avg_cc = np.mean([s.length for s in segs_cc]) if segs_cc else 0
avg_burns = np.mean([s.length for s in segs_burns]) if segs_burns else 0
print(f"Average segment length — LsdCC: {avg_cc:.1f} px, LsdBurns: {avg_burns:.1f} px")

The Burns algorithm typically produces fewer but longer segments because it groups
pixels by gradient direction before fitting, resulting in larger support regions.

### 5.5 LsdEDLZ — Edge Drawing Lines

In [ ]:
det = le_lsd.LsdEDLZ()
det.detect(windmill_gray)
segments = det.line_segments()

canvas = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas, segments)

print(f"LsdEDLZ: {len(segments)} segments")

# Custom parameters
det_custom = le_lsd.LsdEDLZ(gradient_threshold=15.0, anchor_threshold=3.0, min_line_len=10)
det_custom.detect(windmill_gray)
print(f"LsdEDLZ (custom): {len(det_custom.line_segments())} segments")

show_images([windmill_gray, bgr_to_rgb(canvas)],
            ["Original", f"LsdEDLZ ({len(segments)} lines)"])

### 5.6 LsdEL — Edge Linking

In [ ]:
det = le_lsd.LsdEL()
det.detect(windmill_gray)
segments = det.line_segments()

canvas = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas, segments)

print(f"LsdEL: {len(segments)} segments")
print(f"  Flags: EL_USE_NFA={le_lsd.EL_USE_NFA}, EL_USE_PRECISE_SPE={le_lsd.EL_USE_PRECISE_SPE}")

# With NFA validation
det_nfa = le_lsd.LsdEL(flags=le_lsd.EL_USE_NFA)
det_nfa.detect(windmill_gray)
print(f"  With NFA: {len(det_nfa.line_segments())} segments")

show_images([windmill_gray, bgr_to_rgb(canvas)],
            ["Original", f"LsdEL ({len(segments)} lines)"])

### 5.7 LsdEP — Edge Pattern

In [ ]:
det = le_lsd.LsdEP()
det.detect(windmill_gray)
segments = det.line_segments()

canvas = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas, segments)

print(f"LsdEP: {len(segments)} segments")
print(f"  Flag: EP_USE_PRECISE_SPE = {le_lsd.EP_USE_PRECISE_SPE}")

show_images([windmill_gray, bgr_to_rgb(canvas)],
            ["Original", f"LsdEP ({len(segments)} lines)"])

### 5.8 LsdFGioi — Probabilistic LSD (NFA Validation)

In [ ]:
det = le_lsd.LsdFGioi()
det.detect(windmill_gray)
segments = det.line_segments()

canvas = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas, segments)

print(f"LsdFGioi: {len(segments)} segments")

# Custom parameters
det_custom = le_lsd.LsdFGioi(
    quant=3.0, ang_th=30.0, log_eps=1.0, density_th=0.5, n_bins=512
)
det_custom.detect(windmill_gray)
print(f"LsdFGioi (custom): {len(det_custom.line_segments())} segments")

show_images([windmill_gray, bgr_to_rgb(canvas)],
            ["Original", f"LsdFGioi ({len(segments)} lines)"])

### 5.9 LsdHoughP — Probabilistic Hough Transform

In [ ]:
det = le_lsd.LsdHoughP()
det.detect(windmill_gray)
segments = det.line_segments()

canvas = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas, segments)

print(f"LsdHoughP: {len(segments)} segments")

# Custom parameters
det_custom = le_lsd.LsdHoughP(
    th_low=0.01, th_high=0.03, vote_threshold=50, min_length=5.0, max_gap=5.0
)
det_custom.detect(windmill_gray)
print(f"LsdHoughP (custom): {len(det_custom.line_segments())} segments")

show_images([windmill_gray, bgr_to_rgb(canvas)],
            ["Original", f"LsdHoughP ({len(segments)} lines)"])

### 5.10 ValueManager for Detectors

In [ ]:
# Every detector supports runtime parameter inspection and modification
det = le_lsd.LsdCC()
print(f"LsdCC parameters:")
for k, v in det.values().items():
    print(f"  {k}: {v}")

# Modify a parameter
det.set_int("edge_min_pixels", 20)
print(f"\nModified edge_min_pixels: {det.get_value('edge_min_pixels')}")

print("\n" + "="*50)
det2 = le_lsd.LsdFGioi()
print(f"\nLsdFGioi parameters:")
for k, v in det2.values().items():
    print(f"  {k}: {v}")

det2.set_float("angle_th", 25.0)
print(f"\nModified angle_th: {det2.get_value('angle_th')}")

### 5.11 Double Precision (f64) Preset

In [ ]:
# Every detector has a _f64 variant for double-precision computation.
# The input is still uint8, but internal math uses float64.

det_f32 = le_lsd.LsdCC()
det_f32.detect(windmill_gray)
segs_f32 = det_f32.line_segments()

det_f64 = le_lsd.LsdCC_f64()
det_f64.detect(windmill_gray)
segs_f64 = det_f64.line_segments()

print(f"LsdCC (float):  {len(segs_f32)} segments")
print(f"LsdCC (double): {len(segs_f64)} segments")

# Segments from f64 return LineSegment_f64
if segs_f64:
    seg = segs_f64[0]
    print(f"\nFirst f64 segment: type={type(seg).__name__}")
    print(f"  start={seg.start_point()}, end={seg.end_point()}, length={seg.length:.4f}")

---
## 6. Grand Comparison

Run all 9 detectors on the same image and compare results visually and
quantitatively.

In [ ]:
detector_classes = [
    ("LsdCC", le_lsd.LsdCC),
    ("LsdCP", le_lsd.LsdCP),
    ("LsdBurns", le_lsd.LsdBurns),
    ("LsdFBW", le_lsd.LsdFBW),
    ("LsdFGioi", le_lsd.LsdFGioi),
    ("LsdEDLZ", le_lsd.LsdEDLZ),
    ("LsdEL", le_lsd.LsdEL),
    ("LsdEP", le_lsd.LsdEP),
    ("LsdHoughP", le_lsd.LsdHoughP),
]

results = []
images_grid = []
titles_grid = []

for name, cls in detector_classes:
    det = cls()
    
    t = timeit.timeit(lambda: det.detect(windmill_gray), number=3) / 3
    det.detect(windmill_gray)
    segs = det.line_segments()
    
    avg_len = np.mean([s.length for s in segs]) if segs else 0
    results.append((name, len(segs), avg_len, t * 1000))
    
    canvas = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
    le_geometry.draw_lines_random_inplace(canvas, segs)
    images_grid.append(bgr_to_rgb(canvas))
    titles_grid.append(f"{name}\n{len(segs)} segs")

# Summary table
print(f"{'Detector':<12s} {'Segments':>8s} {'Avg Len':>8s} {'Time (ms)':>10s}")
print("-" * 42)
for name, count, avg_len, ms in results:
    print(f"{name:<12s} {count:>8d} {avg_len:>8.1f} {ms:>10.2f}")

# 3x3 grid visualization
show_grid(images_grid, titles_grid, cols=3, figsize=(15, 14))

In [ ]:
# Bar chart comparison
names = [r[0] for r in results]
counts = [r[1] for r in results]
times = [r[3] for r in results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.barh(names, counts, color="steelblue")
ax1.set_xlabel("Number of Segments")
ax1.set_title("Segments Detected")
ax1.invert_yaxis()

ax2.barh(names, times, color="coral")
ax2.set_xlabel("Time (ms)")
ax2.set_title("Detection Time")
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

### Exercise 2.4

Run the grand comparison on a **BSDS500 image** instead of the windmill.
Do the relative rankings change? Create a similar summary table and
identify which detector is fastest and which finds the most segments.

In [ ]:
# TODO: Run all 9 detectors on bsds_gray, print summary table.

**Solution**

In [ ]:
results_bsds = []
for name, cls in detector_classes:
    det = cls()
    t = timeit.timeit(lambda: det.detect(bsds_gray), number=3) / 3
    det.detect(bsds_gray)
    segs = det.line_segments()
    avg_len = np.mean([s.length for s in segs]) if segs else 0
    results_bsds.append((name, len(segs), avg_len, t * 1000))

print(f"{'Detector':<12s} {'Segments':>8s} {'Avg Len':>8s} {'Time (ms)':>10s}")
print("-" * 42)
for name, count, avg_len, ms in results_bsds:
    print(f"{name:<12s} {count:>8d} {avg_len:>8.1f} {ms:>10.2f}")

fastest = min(results_bsds, key=lambda r: r[3])
most = max(results_bsds, key=lambda r: r[1])
print(f"\nFastest: {fastest[0]} ({fastest[3]:.2f} ms)")
print(f"Most segments: {most[0]} ({most[1]} segments)")

The relative rankings can shift depending on image content. Structured scenes
(buildings, roads) tend to favor region-based detectors, while natural scenes
may produce different distributions.

---
## 7. Auxiliary Image Data

Many detectors produce internal data layers (gradient maps, label maps, etc.)
accessible via `.image_data_descriptor()` and `.image_data()`.

In [ ]:
det = le_lsd.LsdCC()
det.detect(windmill_gray)

# Inspect what data layers this detector provides
descriptors = det.image_data_descriptor()
data_layers = det.image_data()

print(f"LsdCC provides {len(descriptors)} data layers:")
for desc, layer in zip(descriptors, data_layers):
    print(f"  '{desc.name}': {desc.description}")
    print(f"    shape={layer.shape}, dtype={layer.dtype}")

# Visualize the data layers
if data_layers:
    show_images(
        [layer.astype(np.float32) if layer.ndim == 2 else layer for layer in data_layers[:4]],
        [desc.name for desc in descriptors[:4]],
    )

In [ ]:
# Compare data descriptors across detectors
for name, cls in [("LsdCC", le_lsd.LsdCC), ("LsdEDLZ", le_lsd.LsdEDLZ),
                   ("LsdFGioi", le_lsd.LsdFGioi)]:
    det = cls()
    det.detect(synth)
    desc = det.image_data_descriptor()
    print(f"{name}: {[d.name for d in desc]}")

---
## 8. Line Optimization

After detection, line segments can be **optimized** by adjusting their position
to better align with the gradient magnitude map. The optimizer minimizes the
perpendicular distance between the segment and strong gradient ridges.

Functions in `le_geometry`:
- `optimize_line_segment(mag, seg)` → `(error, d, r)` — returns new segment
- `optimize_line_segment_inplace(mag, seg)` → `error` — modifies segment
- `optimize_line_segments(mag, segs)` → `(optimized_segs, errors)` — batch

In [ ]:
# Create a synthetic image with a known line for clear demonstration
test_img = np.zeros((100, 100), dtype=np.uint8)
# Draw a line at row 50
test_img[49:51, 10:90] = 255

# Compute gradient magnitude
grad = le_imgproc.SobelGradient()
grad.process(test_img)
mag = grad.magnitude()

# Create a slightly misaligned segment
from_seg = le_lsd.LineSegment.from_endpoints(10.0, 52.0, 89.0, 52.0)
print(f"Before optimization:")
print(f"  start={from_seg.start_point()}, end={from_seg.end_point()}")

# Optimize
err, d, r = le_geometry.optimize_line_segment(mag, from_seg)
print(f"\nOptimization result: error={err:.4f}, d={d:.4f}, r={r:.6f}")

# In-place optimization
seg2 = le_lsd.LineSegment.from_endpoints(10.0, 52.0, 89.0, 52.0)
err2 = le_geometry.optimize_line_segment_inplace(mag, seg2)
print(f"\nAfter in-place optimization:")
print(f"  start={seg2.start_point()}, end={seg2.end_point()}")
print(f"  error={err2:.4f}")

In [ ]:
# Batch optimization on real detected segments
grad = le_imgproc.SobelGradient()
grad.process(windmill_gray)
mag = grad.magnitude()

det = le_lsd.LsdCC()
det.detect(windmill_gray)
segs = det.line_segments()

print(f"Optimizing {len(segs)} segments...")
optimized, errors = le_geometry.optimize_line_segments(mag, segs)

# Statistics
errors_arr = np.array(errors)
print(f"Error range: [{errors_arr.min():.4f}, {errors_arr.max():.4f}]")
print(f"Mean error:  {errors_arr.mean():.4f}")
print(f"Median error: {np.median(errors_arr):.4f}")

# Visualize before vs after
canvas_before = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas_before, segs)

canvas_after = np.zeros((*windmill_gray.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas_after, optimized)

show_images(
    [bgr_to_rgb(canvas_before), bgr_to_rgb(canvas_after)],
    [f"Before ({len(segs)} segs)", f"After optimization ({len(optimized)} segs)"],
)

### Exercise 2.5

1. Detect lines on the windmill using `LsdFGioi`
2. Optimize the detected segments
3. Compute the mean absolute shift in position (difference in center point)
   between original and optimized segments
4. Plot a histogram of the optimization errors

In [ ]:
# TODO: Detect with LsdFGioi, optimize, compute center shifts, plot error histogram.

**Solution**

In [ ]:
grad = le_imgproc.SobelGradient()
grad.process(windmill_gray)
mag = grad.magnitude()

det = le_lsd.LsdFGioi()
det.detect(windmill_gray)
segs_orig = det.line_segments()

optimized, errors = le_geometry.optimize_line_segments(mag, segs_orig)

# Compute center shifts
shifts = []
for orig, opt in zip(segs_orig, optimized):
    cx1, cy1 = orig.center_point()
    cx2, cy2 = opt.center_point()
    shift = np.sqrt((cx2 - cx1)**2 + (cy2 - cy1)**2)
    shifts.append(shift)

print(f"Mean center shift: {np.mean(shifts):.3f} px")
print(f"Max center shift:  {np.max(shifts):.3f} px")

# Error histogram
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.hist(errors, bins=30, color="steelblue", edgecolor="white")
ax1.set_xlabel("Optimization Error")
ax1.set_ylabel("Count")
ax1.set_title("Error Distribution")

ax2.hist(shifts, bins=30, color="coral", edgecolor="white")
ax2.set_xlabel("Center Shift (pixels)")
ax2.set_ylabel("Count")
ax2.set_title("Position Shift Distribution")

plt.tight_layout()
plt.show()

Most segments shift only a fraction of a pixel, indicating good initial detection.
Larger shifts indicate segments that were initially misaligned with the gradient
ridge — optimization corrects this.

---
## 9. Noise Robustness

Real-world images contain noise. Let's examine how detectors behave on
noisy images from the project's noise dataset.

In [ ]:
# Load clean and noisy images from the noise dataset
import cv2

bike_clean = cv2.imread(str(ti.noise("bike.png")), cv2.IMREAD_GRAYSCALE)

noise_levels = ["bike_noise10", "bike_noise20", "bike_noise30", "bike_noise50"]
noisy_images = {}
for name in noise_levels:
    img = cv2.imread(str(ti.noise(f"{name}.png")), cv2.IMREAD_GRAYSCALE)
    noisy_images[name] = img

all_imgs = [bike_clean] + list(noisy_images.values())
all_titles = ["Clean"] + [n.replace("bike_", "") for n in noise_levels]
show_images(all_imgs[:4], all_titles[:4], figsize=(16, 4))

In [ ]:
# Run LsdEL on clean and noisy images
test_images = {"clean": bike_clean}
test_images.update(noisy_images)

detector_names = ["LsdCC", "LsdFGioi", "LsdEL"]

print(f"{'Image':<16s}", end="")
for dname in detector_names:
    print(f"{dname:>10s}", end="")
print()
print("-" * (16 + 10 * len(detector_names)))

noise_results = {dname: [] for dname in detector_names}

for img_name, img in test_images.items():
    print(f"{img_name:<16s}", end="")
    for dname in detector_names:
        cls = getattr(le_lsd, dname)
        det = cls()
        det.detect(img)
        n = len(det.line_segments())
        noise_results[dname].append(n)
        print(f"{n:>10d}", end="")
    print()

In [ ]:
# Visualize degradation: LsdEL on clean vs noisiest
det_clean = le_lsd.LsdEL()
det_clean.detect(bike_clean)
segs_clean = det_clean.line_segments()

det_noisy = le_lsd.LsdEL()
det_noisy.detect(noisy_images["bike_noise50"])
segs_noisy = det_noisy.line_segments()

canvas_c = np.zeros((*bike_clean.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas_c, segs_clean)

canvas_n = np.zeros((*bike_clean.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas_n, segs_noisy)

show_images(
    [bgr_to_rgb(canvas_c), bgr_to_rgb(canvas_n)],
    [f"Clean ({len(segs_clean)} segs)", f"Noise50 ({len(segs_noisy)} segs)"],
)

In [ ]:
# Plot noise level vs segment count
x_labels = list(test_images.keys())

plt.figure(figsize=(10, 5))
for dname in detector_names:
    plt.plot(x_labels, noise_results[dname], "o-", label=dname)

plt.xlabel("Image")
plt.ylabel("Number of Segments")
plt.title("Noise Robustness: Segments Detected")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Exercise 2.6

Investigate whether **raising the NMS thresholds** helps detectors deal with noise:

1. Use `LsdCC` with default thresholds on `bike_noise30`
2. Use `LsdCC` with `th_low=0.02, th_high=0.06` on the same noisy image
3. Compare the number and average length of detected segments
4. Do higher thresholds reduce false detections (short, spurious segments)?

In [ ]:
# TODO: Compare LsdCC default vs high thresholds on noisy image.

**Solution**

In [ ]:
noisy = noisy_images["bike_noise30"]

det_default = le_lsd.LsdCC()
det_default.detect(noisy)
segs_def = det_default.line_segments()

det_high = le_lsd.LsdCC(th_low=0.02, th_high=0.06)
det_high.detect(noisy)
segs_high = det_high.line_segments()

avg_def = np.mean([s.length for s in segs_def]) if segs_def else 0
avg_high = np.mean([s.length for s in segs_high]) if segs_high else 0

short_def = sum(1 for s in segs_def if s.length < 10)
short_high = sum(1 for s in segs_high if s.length < 10)

print(f"Default thresholds: {len(segs_def)} segs, avg length {avg_def:.1f}, short (<10px): {short_def}")
print(f"High thresholds:    {len(segs_high)} segs, avg length {avg_high:.1f}, short (<10px): {short_high}")

canvas_def = np.zeros((*noisy.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas_def, segs_def)

canvas_high = np.zeros((*noisy.shape, 3), dtype=np.uint8)
le_geometry.draw_lines_random_inplace(canvas_high, segs_high)

show_images(
    [bgr_to_rgb(canvas_def), bgr_to_rgb(canvas_high)],
    [f"Default ({len(segs_def)} segs)", f"High threshold ({len(segs_high)} segs)"],
)

Higher thresholds suppress noise-induced weak edges, reducing the number of
short spurious segments. The trade-off is that some genuine weak edges may
also be lost.

---
## Summary

In this tutorial you learned:

- **Edge Sources**: Three gradient operators with built-in NMS and hysteresis
- **Standalone NMS**: Manual control over non-maximum suppression parameters
- **Edge Segment Detectors**: Four algorithms for tracing edge contours (Simple,
  Drawing, Linking, Pattern) with varying gap-bridging capabilities
- **Line Segment Detectors**: All 9 LSD algorithms — their parameters, flags,
  and trade-offs between speed, accuracy, and noise robustness
- **Auxiliary Data**: Accessing internal detector data layers for analysis
- **Line Optimization**: Refining detected segments against gradient magnitude
- **Noise Robustness**: How noise affects detection and threshold tuning

### Next Steps

Continue with **Tutorial 3: Performance Evaluation Framework** to learn how
to benchmark and systematically compare detectors across image datasets.